In [46]:
# Create search engine!
# Goodreads json file is too large to open entirely using a pandas Dataframe. Instead, we first read it line by line:
import gzip
with gzip.open("goodreads_books.json.gz", 'r') as f:
    line = f.readline()

In [47]:
import json

json.loads(line)

{'isbn': '0312853122',
 'text_reviews_count': '1',
 'series': [],
 'country_code': 'US',
 'language_code': '',
 'popular_shelves': [{'count': '3', 'name': 'to-read'},
  {'count': '1', 'name': 'p'},
  {'count': '1', 'name': 'collection'},
  {'count': '1', 'name': 'w-c-fields'},
  {'count': '1', 'name': 'biography'}],
 'asin': '',
 'is_ebook': 'false',
 'average_rating': '4.00',
 'kindle_asin': '',
 'similar_books': [],
 'description': '',
 'format': 'Paperback',
 'link': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'authors': [{'author_id': '604031', 'role': ''}],
 'publisher': "St. Martin's Press",
 'num_pages': '256',
 'publication_day': '1',
 'isbn13': '9780312853129',
 'publication_month': '9',
 'edition_information': '',
 'publication_year': '1984',
 'url': 'https://www.goodreads.com/book/show/5333265-w-c-fields',
 'image_url': 'https://images.gr-assets.com/books/1310220028m/5333265.jpg',
 'book_id': '5333265',
 'ratings_count': '3',
 'work_id': '5400751',
 'title': '

In [48]:
# Get most important fields from the goodreads json file. These fields include the book ID, title, number of ratings, url, and cover (url).
def parse_books(line):
    book = json.loads(line)
    return {
        "book_id": book["book_id"],
        "title": book["title_without_series"],
        "ratings": book["ratings_count"],
        "url": book["url"],
        "cover": book["image_url"]
    }

In [49]:
# Narrow down search by only including books for which there are at least 20 ratings. (Books with less than 20 ratings are obviously not very popular and thus are not necessary inclusions in our search engine).
# Append all of the books that match these requirements into a list called title.
titles = []
with gzip.open("goodreads_books.json.gz", 'r') as f:
    while True:
        line = f.readline()
        if not line:
            break
        fields = parse_books(line)
        
        try:
            ratings = int(fields["ratings"])
        except ValueError:
            continue
        if ratings > 20:
            titles.append(fields)

In [50]:
# NOW we can use a pandas DataFrame
import pandas as pd

book_titles = pd.DataFrame.from_dict(titles)

In [51]:
book_titles["ratings"] = pd.to_numeric(book_titles["ratings"])

book_titles["modified_title"] = book_titles["title"].str.replace("[^a-zA-Z0-9 ]", "", regex=True) #this is a regular expression that modifies titles so that any that include characters other than 
                                                                                                #those in the brackets are removed.

In [52]:
book_titles["modified_title"] = book_titles["modified_title"].str.lower()

In [53]:
book_titles["modified_title"] = book_titles["modified_title"].str.replace("\s+", " ", regex=True) # replace multiple spaces with one space

In [54]:
book_titles = book_titles[book_titles["modified_title"].str.len() > 0] # removing blank titles

In [55]:
book_titles.to_json("book_titles.json")

In [34]:
book_titles

,book_id,title,ratings,url,cover,modified_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook
...,...,...,...,...,...,...
1171886,1370179,The Brazilian Boss's Innocent Mistress,240,https://www.goodreads.com/book/show/1370179.Th...,https://s.gr-assets.com/assets/nophoto/book/11...,the brazilian bosss innocent mistress
1171887,17805813,"Ondine (Ondine Quartet, #0.5)",327,https://www.goodreads.com/book/show/17805813-o...,https://images.gr-assets.com/books/1379766592m...,ondine ondine quartet 05
1171888,2685097,The Spaniard's Blackmailed Bride,112,https://www.goodreads.com/book/show/2685097-th...,https://s.gr-assets.com/assets/nophoto/book/11...,the spaniards blackmailed bride
1171889,2342551,The Children's Classic Poetry Collection,36,https://www.goodreads.com/book/show/2342551.Th...,https://s.gr-assets.com/assets/nophoto/book/11...,the childrens classic poetry collection
